# Extracting rPPG Signal with DeepPhys

In [ ]:
import respiration.dataset as repository

dataset = repository.from_default()

subject = 'Proband05'
setting = '101_natural_lighting'

In [ ]:
frames, meta = dataset.get_video_bgr(subject, setting, show_progress=True)

In [ ]:
import respiration.utils as utils

device = utils.get_torch_device()

In [ ]:
import torch
from respiration.extractor.deep_phys import DeepPhys

model_path = '../../data/rPPG-Toolbox/BP4D_PseudoLabel_DeepPhys.pth'
# model_path = '../../data/rPPG-Toolbox/PURE_DeepPhys.pth'
# model_path = '../../data/rPPG-Toolbox/UBFC-rPPG_DeepPhys.pth'
# model_path = '../../data/rPPG-Toolbox/MA-UBFC_deepphys.pth'
# model_path = '../../data/rPPG-Toolbox/SCAMPS_DeepPhys.pth'

# Wrap modul in nn.DataParallel
model = DeepPhys(img_size=72)
# Fix model loading: Some key have an extra 'module.' prefix
model = torch.nn.DataParallel(model)
model.to(device)

# Load the model with the weights
key_matching = model.load_state_dict(torch.load(model_path, map_location=device))
key_matching

In [ ]:
model.eval()

In [ ]:
from respiration.extractor.mtts_can import (
    preprocess_video_frames,
    preprocess_frames_original,
)

# resized, normalized = preprocess_frames_original(frames)
resized, normalized = preprocess_video_frames(frames, dim=72)

In [ ]:
resized.shape, normalized.shape

In [ ]:
import matplotlib.pyplot as plt

# Plot resized and normalized frames
_, axs = plt.subplots(1, 2, figsize=(15, 5))

axs[0].imshow(resized[1])
axs[0].set_title('Resized Frame')

axs[1].imshow(normalized[1])
axs[1].set_title('Normalized Frame')

In [ ]:
# Convert the frames to a tensor
resized_tensor = torch.tensor(resized, device=device)
normalized_tensor = torch.tensor(normalized, device=device)

In [ ]:
resized_tensor.shape, normalized_tensor.shape

In [ ]:
# Get the first 10 seconds of the video
chunk_resized = resized_tensor[:500]
chunk_normalized = normalized_tensor[:500]

# Transform the tensor to the shape expected by the model (frame_count, c, w, h)
chunk_resized = chunk_resized.permute(0, 3, 1, 2)
chunk_normalized = chunk_normalized.permute(0, 3, 1, 2)

In [ ]:
chunk_resized.shape

In [ ]:
chunk_resized.shape, chunk_normalized.shape

In [ ]:
# Combine chunk_resized and chunk_normalized into a single tensor
chunk = torch.cat((chunk_resized, chunk_normalized), dim=1)
chunk.shape

In [ ]:
# Extract the rPPG signal
with torch.no_grad():
    rppg = model(chunk)

rppg.shape

In [ ]:
import respiration.preprocessing as preprocessing

waveform = rppg.cpu().numpy()
waveform = preprocessing.detrend_tarvainen(waveform)

In [ ]:
# Plot the rPPG signal
plt.figure(figsize=(15, 5))
plt.plot(waveform)
plt.title('rPPG Signal')
plt.xlabel('Frame')
plt.ylabel('Amplitude')
plt.show()